## 🧪 Example: Simple Kafka Producer with Message Counting

In [2]:

from confluent_kafka import Producer
import time
import threading


In [4]:

class CountingProducer:
    def __init__(self, config, window_seconds=10):
        self.producer = Producer(config)
        self.sent_count = 0
        self.ack_count = 0
        self.lock = threading.Lock()
        self.window_seconds = window_seconds
        self._start_window_timer()

    def _start_window_timer(self):
        def reset_counts():
            while True:
                time.sleep(self.window_seconds)
                with self.lock:
                    print(f"[{time.strftime('%X')}] Sent: {self.sent_count}, Acks: {self.ack_count}")
                    self.sent_count = 0
                    self.ack_count = 0
        threading.Thread(target=reset_counts, daemon=True).start()

    def _delivery_report(self, err, msg):
        with self.lock:
            if err is None:
                self.ack_count += 1
            else:
                print(f"Delivery failed: {err}")

    def produce(self, topic, value, key=None):
        with self.lock:
            self.sent_count += 1
        self.producer.produce(topic=topic, value=value, key=key, callback=self._delivery_report)
        self.producer.poll(0)

    def flush(self):
        self.producer.flush()


### Example usage


In [5]:

if __name__ == "__main__":
    config = {
        'bootstrap.servers': '10.222.68.223:9092'
    }

    cp = CountingProducer(config, window_seconds=10)

    for i in range(100):
        cp.produce('test-topic', value=f'message-{i}')
        time.sleep(0.1)  # simulate delay between messages

    cp.flush()


[13:27:17] Sent: 100, Acks: 99


[13:27:27] Sent: 0, Acks: 1
[13:27:37] Sent: 0, Acks: 0
[13:27:47] Sent: 0, Acks: 0
[13:27:57] Sent: 0, Acks: 0
[13:28:07] Sent: 0, Acks: 0
[13:28:17] Sent: 0, Acks: 0
[13:28:27] Sent: 0, Acks: 0
